<a href="https://colab.research.google.com/github/pavankalyantummala/srl/blob/master/SRLGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

data = pd.read_csv("/content/drive/My Drive/SRL/fulltrainmod(5).csv", encoding="latin1")
data = data.fillna(method="ffill")
data.tail(20)

,Sentence,word,tag,pred
4625280,Sentence: 167346,",",O,PADword
4625281,Sentence: 167346,the,O,PADword
4625282,Sentence: 167346,board,O,PADword
4625283,Sentence: 167346,might,O,PADword
4625284,Sentence: 167346,be,O,PADword
4625285,Sentence: 167346,force,O,PADword
4625286,Sentence: 167346,to,A2,PADword
4625287,Sentence: 167346,exclude,O,PADword
4625288,Sentence: 167346,him,O,PADword
4625289,Sentence: 167346,from,O,PADword


In [0]:
data1 = pd.read_csv("/content/drive/My Drive/SRL/fullevalumod.csv", encoding="latin1")
data1 = data1.fillna(method="ffill")
data1.tail(20)

,Sentence,word,tag,pred
281457,Sentence: 10081,bolster,O,bolster
281458,Sentence: 10081,its,O,PADword
281459,Sentence: 10081,raw,O,PADword
281460,Sentence: 10081,material,O,PADword
281461,Sentence: 10081,supply,A1,PADword
281462,Sentence: 10081,and,O,PADword
281463,Sentence: 10081,develop,O,PADword
281464,Sentence: 10081,new,O,PADword
281465,Sentence: 10081,use,O,PADword
281466,Sentence: 10081,for,O,PADword


In [0]:
words = set(list(data['word'].values))
words.add('PADword')
n_words = len(words)
n_words

28377

In [0]:
tags = list(set(data["tag"].values))
n_tags = len(tags)
n_tags

53

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["tag"].values.tolist(),
                                                           s["pred"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

[('ms.', 'O', 'PADword'), ('haag', 'A0', 'PADword'), ('play', 'O', 'play'), ('elianti', 'A1', 'PADword'), ('.', 'O', 'PADword')]


In [0]:
getter1 = SentenceGetter(data1)
sent1 = getter1.get_next()
print(sent1)

[('but', 'O', 'PADword'), ('while', 'O', 'PADword'), ('the', 'O', 'PADword'), ('new', 'O', 'PADword'), ('york', 'O', 'PADword'), ('stock', 'O', 'PADword'), ('exchange', 'A1', 'PADword'), ('do', 'O', 'PADword'), ('not', 'AM-NEG', 'PADword'), ('fall', 'O', 'fall'), ('apart', 'A4', 'PADword'), ('friday', 'AM-TMP', 'PADword'), ('as', 'AM-TMP', 'PADword'), ('the', 'O', 'PADword'), ('dow', 'O', 'PADword'), ('jones', 'O', 'PADword'), ('industrial', 'O', 'PADword'), ('average', 'O', 'PADword'), ('plunge', 'O', 'PADword'), ('190.58', 'O', 'PADword'), ('point', 'O', 'PADword'), ('--', 'O', 'PADword'), ('most', 'O', 'PADword'), ('of', 'O', 'PADword'), ('it', 'O', 'PADword'), ('in', 'O', 'PADword'), ('the', 'O', 'PADword'), ('final', 'O', 'PADword'), ('hour', 'O', 'PADword'), ('--', 'O', 'PADword'), ('it', 'O', 'PADword'), ('barely', 'O', 'PADword'), ('manage', 'O', 'PADword'), ('to', 'O', 'PADword'), ('stay', 'O', 'PADword'), ('this', 'O', 'PADword'), ('side', 'O', 'PADword'), ('of', 'O', 'PADwor

In [0]:
sentences = getter.sentences
print(len(sentences))
sentences1 = getter1.sentences
print(len(sentences1))

167347
10074


In [0]:
sentences2=[]
for i in range(len(sentences1)):
  k=0
  for j in range(len(sentences1[i])):
    if(sentences1[i][j][2]!='PADword'):
      k=1
  if(k==1):
    sentences2.append(sentences1[i][:])

In [0]:
largest_sen = max(len(sen) for sen in sentences)
print('biggest sentence has {} words'.format(largest_sen))

biggest sentence has 143 words


In [0]:
tags2index = {t:i for i,t in enumerate(tags)}
print(tags2index['A0'])

14


In [0]:
tags2index

{'A0': 3,
 'A1': 49,
 'A2': 4,
 'A3': 10,
 'A4': 16,
 'A5': 21,
 'AA': 44,
 'AM': 18,
 'AM-ADV': 13,
 'AM-CAU': 22,
 'AM-DIR': 9,
 'AM-DIS': 38,
 'AM-EXT': 29,
 'AM-LOC': 7,
 'AM-MNR': 19,
 'AM-MOD': 41,
 'AM-NEG': 11,
 'AM-PNC': 47,
 'AM-PRD': 25,
 'AM-PRT': 40,
 'AM-REC': 34,
 'AM-TM': 2,
 'AM-TMP': 33,
 'C-A0': 42,
 'C-A1': 5,
 'C-A2': 32,
 'C-A3': 20,
 'C-A4': 27,
 'C-AM-ADV': 45,
 'C-AM-CAU': 39,
 'C-AM-DIR': 1,
 'C-AM-DIS': 14,
 'C-AM-EXT': 6,
 'C-AM-LOC': 24,
 'C-AM-MNR': 17,
 'C-AM-NEG': 31,
 'C-AM-PNC': 0,
 'C-AM-TMP': 46,
 'O': 51,
 'R-A0': 28,
 'R-A1': 30,
 'R-A2': 36,
 'R-A3': 48,
 'R-A4': 43,
 'R-AA': 8,
 'R-AM-ADV': 12,
 'R-AM-CAU': 35,
 'R-AM-DIR': 50,
 'R-AM-EXT': 26,
 'R-AM-LOC': 52,
 'R-AM-MNR': 37,
 'R-AM-PNC': 15,
 'R-AM-TMP': 23}

In [0]:
import pickle
with open("/content/drive/My Drive/SRL/tag2index.pkl", "wb") as encoded_pickle:
    pickle.dump(tags2index, encoded_pickle)

In [0]:
from pickle import load 
tags2index = load(open("/content/drive/My Drive/SRL/tag2index.pkl", "rb"))
print('Tags: train=%d' % len(tags2index))

Tags: train=53


In [0]:
max_len = 64
X = [[w[0]for w in s] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)
new_X[15]

['after',
 'the',
 'confusion',
 'surround',
 'the',
 'change',
 'of',
 'the',
 'coke',
 'formula',
 'in',
 '1985',
 ',',
 'coca',
 '-',
 'cola',
 'be',
 'reluctant',
 'to',
 'clutter',
 'the',
 'classic',
 'name',
 'with',
 'a',
 'brand',
 'extension',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword']

In [0]:
max_len = 64
X = [[w[0]for w in s] for s in sentences2]
new_Xtest = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
          if(seq[i] in word_index):
                        new_seq.append(seq[i])
          else:
            new_seq.append('PADword')
        except:
            new_seq.append("PADword")
    new_Xtest.append(new_seq)
new_Xtest[13]

['the',
 'rating',
 'concern',
 'say',
 'franklin',
 "'s",
 '``',
 'trouble',
 'diversification',
 'record',
 'in',
 'the',
 'security',
 'business',
 "''",
 'be',
 'one',
 'reason',
 'for',
 'the',
 'downgrade',
 ',',
 'cite',
 'the',
 'trouble',
 'at',
 'its',
 'PADword',
 'rothschild',
 'subsidiary',
 'and',
 'the',
 'possible',
 'sale',
 'of',
 'other',
 'subsidiary',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword']

In [0]:
max_len = 64
X = [[w[2]for w in s] for s in sentences]
new_P = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_P.append(new_seq)
new_P[15]

['PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'formula',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword']

In [0]:
max_len = 64
X = [[w[2]for w in s] for s in sentences2]
new_Ptest = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            if(seq[i] in word_index):
                          new_seq.append(seq[i])
            else:
              new_seq.append("PADword")
        except:
            new_seq.append("PADword")
    new_Ptest.append(new_seq)
new_Ptest[15]

['PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'secretary',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword',
 'PADword']

In [0]:
!unzip -uq "/content/drive/My Drive/SRL/syngcn_embed.zip" -d "/content/drive/My Drive/SRL/"

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle


Using TensorFlow backend.


In [0]:
features = n_words
tokenizer = Tokenizer(num_words = features)
tokenizer.fit_on_texts(new_X)
word_index = tokenizer.word_index
newx = tokenizer.texts_to_sequences(new_X)

In [0]:

# saving
with open('/content/drive/My Drive/SRL/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:

with open('/content/drive/My Drive/SRL/tokenizer.pickle', 'rb') as handle:
    tokenizer1 = pickle.load(handle)

In [0]:
word_index = tokenizer1.word_index
newx = tokenizer1.texts_to_sequences(new_X)

In [0]:
np.array(newx).shape

(167347, 64)

In [0]:
newp = tokenizer1.texts_to_sequences(new_P)

In [0]:
np.array(newp).shape

(167347, 64)

In [0]:
newxte = tokenizer1.texts_to_sequences(new_Xtest)


In [0]:
np.array(newxte).shape

(8011, 64)

In [0]:
newpte = tokenizer1.texts_to_sequences(new_Ptest)

In [0]:
np.array(newpte).shape

(8011, 64)

In [0]:
word_index['the']

2

In [0]:
from keras.preprocessing.sequence import pad_sequences
y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])
y[15]

array([51, 51, 51, 51, 51, 51, 51, 51,  3, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51], dtype=int32)

In [0]:
ytest = [[tags2index[w[1]] for w in p] for p in sentences2]
ytest = pad_sequences(maxlen=max_len, sequences=ytest, padding="post", value=tags2index["O"])
ytest[15]

array([51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,  3,  4, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51], dtype=int32)

In [0]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te,pred_tr,pred_te = train_test_split(newx, y, newp,test_size=0.01, random_state=2018)

In [0]:
print(np.array(ytest).shape)

(10074, 64)


In [0]:
import os
GLOVE_DIR=''
embeddings = {}
k=0
with open(os.path.join(GLOVE_DIR,'/content/drive/My Drive/SRL/syngcn_embeddings.txt')) as f:
    for line in f:
        try:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:])
            em = coefs.astype(np.float)
            k=k+1
            embeddings[word] = em
        except:
            print("hii")

len(embeddings)


hii
hii
hii
hii
hii
hii


149994

In [0]:
num_words = min(n_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, 300))
for word, i in word_index.items():
    if i >= n_words:
        continue
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix.shape

(28353, 300)

In [0]:
batch_size = 64
import tensorflow as tf
from keras import backend as K
from keras.models import Model, Input
from keras.layers.merge import concatenate
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from keras.initializers import Constant
pretrained_embedding_layer = Embedding(num_words,
                                300,
                                embeddings_initializer=Constant(embedding_matrix),
                                input_length=max_len,
                                trainable=False)

In [0]:
print(np.array(newx).shape)
print(np.array(newxte).shape)
print(np.array(newp).shape)
print(np.array(newpte).shape)
print(np.array(y).shape)
print(np.array(ytest).shape)

(167347, 64)
(8011, 64)
(167347, 64)
(8011, 64)
(167347, 64, 1)
(8011, 64, 1)


In [0]:
y = y.reshape(y.shape[0], y.shape[1], 1)
ytest = ytest.reshape(ytest.shape[0], ytest.shape[1], 1)

In [0]:
np.array(newx).shape

(167347, 64)

In [0]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = pretrained_embedding_layer(sequence_input)
sequence_input1 = Input(shape=(max_len,), dtype='int32')
embedded_sequences1 = pretrained_embedding_layer(sequence_input1)
t = concatenate([embedded_sequences, embedded_sequences1])
x = Bidirectional(LSTM(units=64, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(t)
x_rnn = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x_rnn)
model = Model([sequence_input,sequence_input1], out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 64, 300)      8505900     input_17[0][0]                   
                                                                 input_18[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 64, 600)      0           embedding_4[4][0]          

In [0]:
history = model.fit([newx,newp], y, validation_data=([newxte,newpte],ytest),
                    batch_size=32, epochs=20, verbose=1)

Train on 167347 samples, validate on 8011 samples
Epoch 1/30
167347/167347 [==============================] - 1491s 9ms/step - loss: 0.0908 - val_loss: 0.0659
Epoch 2/30
167347/167347 [==============================] - 1490s 9ms/step - loss: 0.0620 - val_loss: 0.0565
Epoch 3/30
167347/167347 [==============================] - 1495s 9ms/step - loss: 0.0547 - val_loss: 0.0523
Epoch 4/30
167347/167347 [==============================] - 1496s 9ms/step - loss: 0.0509 - val_loss: 0.0501
Epoch 5/30
167347/167347 [==============================] - 1497s 9ms/step - loss: 0.0484 - val_loss: 0.0485
Epoch 6/30
167347/167347 [==============================] - 1492s 9ms/step - loss: 0.0466 - val_loss: 0.0471
Epoch 7/30
167347/167347 [==============================] - 1493s 9ms/step - loss: 0.0453 - val_loss: 0.0466
Epoch 8/30
167347/167347 [==============================] - 1505s 9ms/step - loss: 0.0443 - val_loss: 0.0460
Epoch 9/30
167347/167347 [==============================] - 1495s 9ms/step - l

In [0]:
model.save_weights("/content/drive/My Drive/SRL/model30ep.h5")

In [0]:
print(history.history['loss'])

[0.09851549382959082]


In [0]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=de3efe7b1eeed4409eadfdfcddb78abfff2a435dcf805e92bd8e23474bc2022c
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = mode.predict([np.array(newxte),np.array(newpte)], verbose=1)

8011/8011 [==============================] - 18s 2ms/step


In [0]:
idx2tag = {i: w for w, i in tags2index.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p].replace("PADword", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = test2label(ytest[:,:,0])

In [0]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 77.0%


In [0]:
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

       A1       0.82      0.81      0.82      6737
       A0       0.80      0.79      0.79      4519
      ADV       0.61      0.47      0.53       394
      LOC       0.61      0.58      0.59       463
       A2       0.72      0.66      0.69      2024
      MOD       0.93      0.97      0.95       425
       A3       0.78      0.59      0.67       388
      NEG       0.94      0.92      0.93       178
      MNR       0.61      0.65      0.63       539
      DIS       0.60      0.70      0.65       181
      TMP       0.76      0.75      0.75      1169
      EXT       0.59      0.52      0.55        58
      PNC       0.47      0.56      0.51        85
      DIR       0.68      0.45      0.54        62
       A4       0.76      0.80      0.78        90
      PRD       0.00      0.00      0.00         3
       A5       0.67      0.40      0.50         5
      CAU       0.64      0.71      0.67        51
      REC       0.00      0.00

In [0]:
idx2tag

{0: 'C-AM-PNC',
 1: 'C-AM-DIR',
 2: 'AM-TM',
 3: 'A0',
 4: 'A2',
 5: 'C-A1',
 6: 'C-AM-EXT',
 7: 'AM-LOC',
 8: 'R-AA',
 9: 'AM-DIR',
 10: 'A3',
 11: 'AM-NEG',
 12: 'R-AM-ADV',
 13: 'AM-ADV',
 14: 'C-AM-DIS',
 15: 'R-AM-PNC',
 16: 'A4',
 17: 'C-AM-MNR',
 18: 'AM',
 19: 'AM-MNR',
 20: 'C-A3',
 21: 'A5',
 22: 'AM-CAU',
 23: 'R-AM-TMP',
 24: 'C-AM-LOC',
 25: 'AM-PRD',
 26: 'R-AM-EXT',
 27: 'C-A4',
 28: 'R-A0',
 29: 'AM-EXT',
 30: 'R-A1',
 31: 'C-AM-NEG',
 32: 'C-A2',
 33: 'AM-TMP',
 34: 'AM-REC',
 35: 'R-AM-CAU',
 36: 'R-A2',
 37: 'R-AM-MNR',
 38: 'AM-DIS',
 39: 'C-AM-CAU',
 40: 'AM-PRT',
 41: 'AM-MOD',
 42: 'C-A0',
 43: 'R-A4',
 44: 'AA',
 45: 'C-AM-ADV',
 46: 'C-AM-TMP',
 47: 'AM-PNC',
 48: 'R-A3',
 49: 'A1',
 50: 'R-AM-DIR',
 51: 'O',
 52: 'R-AM-LOC'}

In [0]:
i = 67
p = mode.predict([np.array(newxte[i:i+32]),np.array(newpte[i:i+32])])[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(newxte[i], ytest[i,:,0], p):
    if w != 1:
        listofkeys = [key for (key, value) in word_index.items() if value == w ]
        print("{:15}:{:5} ({})".format(listofkeys[0], idx2tag[pred], idx2tag[true]))
k=0
while k<len(newpte[i]):
    if(newpte[i][k]!=1):
        listofkeys = [key for (key, value) in word_index.items() if value == newpte[i][k] ]
        print("\nPredicate: "+ listofkeys[0])
        break
    k=k+1

Word            Pred : (True)
toshiba        :O     (O)
's             :O     (O)
line           :O     (O)
of             :O     (O)
portable       :O     (O)
,              :O     (O)
for            :O     (O)
example        :O     (O)
,              :O     (O)
feature        :O     (O)
the            :O     (O)
t              :O     (O)
-              :O     (O)
,              :O     (O)
which          :O     (O)
be             :O     (O)
in             :O     (O)
the            :O     (O)
same           :O     (O)
weight         :O     (O)
class          :O     (O)
but            :O     (O)
be             :O     (O)
much           :O     (O)
slow           :O     (O)
and            :O     (O)
have           :O     (O)
less           :O     (O)
memory         :O     (O)
,              :O     (O)
and            :O     (O)
the            :O     (O)
t              :O     (O)
-              :O     (O)
,              :O     (O)
which          :O     (O)
also           :O     (O)
use     

In [0]:
from keras.models import load_model


In [0]:
model.save("/content/drive/My Drive/SRL/mode30.h5")

In [0]:
mode=load_model("/content/drive/My Drive/SRL/mode30.h5")

In [0]:
mode.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 64, 300)      8505900     input_17[0][0]                   
                                                                 input_18[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 64, 600)      0           embedding_4[0][0]          

In [0]:
mode.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [0]:
history3 = mode.fit([newx,newp], y, validation_data=([newxte,newpte],ytest),
                    batch_size=32, epochs=2, verbose=1)

Train on 167347 samples, validate on 8011 samples
Epoch 1/2
167347/167347 [==============================] - 1731s 10ms/step - loss: 0.0389 - val_loss: 0.0424
Epoch 2/2
167347/167347 [==============================] - 1726s 10ms/step - loss: 0.0387 - val_loss: 0.0425


In [0]:
mode.save("/content/drive/My Drive/SRL/modeext.h5")

In [0]:
inp=input("Sentence:" )
predicate=input("Predicate: ")

max_len = 64
X = [s for s in inp.split()]
usrinp = []
new_seq = []
for i in range(max_len):
        try:
            if(X[i] in word_index):
                          new_seq.append(X[i])
            else:
              new_seq.append("PADword")
        except:
            new_seq.append("PADword")
usrinp.append(new_seq)
userpad=[]
for i in range(max_len):
  if(usrinp[0][i]==predicate):
    userpad.append(predicate)
  else:
    userpad.append("PADword")
userte = tokenizer1.texts_to_sequences(usrinp)
usrpred= tokenizer1.texts_to_sequences([userpad])
x=inp.split()
p = mode.predict([userte,usrpred])[0]
p = np.argmax(p, axis=-1)
print("\n")
wr=0
for pred in p:
        print(x[wr],idx2tag[pred])
        wr+=1
        if(wr==len(inp.split())):
          break
k=0


Sentence:mary cooked the food
Predicate: cooked


mary A0
cooked O
the O
food A1
